<a href="https://colab.research.google.com/github/einsrojas08/KNavigator/blob/main/KNavigator_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [16]:
import pandas as pd
import numpy as np
import os
import random
import matplotlib.pyplot as plt

SyntaxError: ignored

In [19]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 78kB/s 
     |████████████████████████████████| 204kB 45.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=24f6b8b9394dd738f5dd55e0bf84f7d284f2c840c011bcd8ef8287ec7f30a3d5
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext(master="local[2]")

In [40]:
import pyspark.ml.feature
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF
from pyspark.ml.feature import StringIndexer

In [4]:
#check files
data_dir = '/content/drive/MyDrive/Kyosei Navigator'
os.listdir(data_dir)

['ORGdataset.csv',
 'randomillness.csv',
 'classifiedpatients.csv',
 'KyoseiNavigator.ipynb',
 'KNavigator-Classifier.ipynb']

In [8]:
#read columns
dfcp = pd.read_csv('/content/drive/MyDrive/Kyosei Navigator/classifiedpatients.csv')
print(dfcp.columns)

Index(['Patient ID', 'Illness'], dtype='object')


In [10]:
#read columns
dforg = pd.read_csv('/content/drive/MyDrive/Kyosei Navigator/ORGdataset.csv')
print(dforg.columns)

Index(['Name of Organization', 'Specific Disease', 'Email '], dtype='object')


In [24]:
spark = SparkSession.builder.appName("orgclassified").getOrCreate()

In [67]:
dfcp = spark.read.csv('/content/drive/MyDrive/Kyosei Navigator/classifiedpatients.csv', header=True, inferSchema=True)
dfcp.select('Patient ID','Illness')
dfcp.columns

['Patient ID', 'Illness']

In [ ]:
dfcp.groupby('Illness').count().show()

In [71]:
tokenizer = Tokenizer(inputCol='Illness',outputCol='mytokens')
stopwords_remover = StopWordsRemover(inputCol='mytokens',outputCol='filtered_tokens')
vectorizer = CountVectorizer(inputCol='filtered_tokens',outputCol='rawFeatures')
idf = IDF(inputCol='rawFeatures',outputCol='vectorizedFeatures')

In [72]:
labelEncoder = StringIndexer(inputCol='Illness',outputCol='label').fit(dfcp)

In [106]:
labelEncoder.transform(dfcp).show(5)

+----------+--------------------+-----+
|Patient ID|             Illness|label|
+----------+--------------------+-----+
|         1|Children with Chi...|  2.0|
|         2|       Throat Cancer|  6.0|
|         3|Women with Cervic...| 10.0|
|         4|     Thyroid Cancer |  3.0|
|         5|Patients with Dia...|  5.0|
+----------+--------------------+-----+
only showing top 5 rows



In [76]:
labelEncoder.labels

['Depression',
 'Blood Disorder',
 'Children with Childhood Cancer',
 'Thyroid Cancer ',
 'Patients having sideway curvature of spine',
 'Patients with Diabetes',
 'Throat Cancer',
 'Children with cancer',
 'Chronic Kidney Disease',
 'Nueromuscular Disease',
 'Women with Cervical Cancer',
 ' Type 1 Diabetes Patients',
 'Disease that damage and weakens the muscle',
 'Genetic Disorder of Nervous System',
 'Groups of Men Surviving HIV',
 'Groups of Mothers of Children with Acute Myeloid Leukemia',
 'Groups of Surviving Women with HIV',
 'Patient Suffering with Arthritis',
 'Patient with Hepatitis',
 'Patients with Psoriasis',
 'Patients with Rheumatoid',
 'Ballooning of blood vessel in the brain',
 'Caring for Senior Citizen',
 'Chronic Myeloid Leukemia',
 'Disoder on muscle and tone posture causing damage to brain',
 'Hearing Disability',
 'Lack of sufficient blood Clothing',
 'Skin Diseases with too much protein collagen',
 'Type of Arthritis with a long term inflammation of joints and 

In [107]:
label_dict = {'Depression':0.0,
 'Blood Disorder':1.0,
 'Children with Childhood Cancer':2.0,
 'Thyroid Cancer ':3.0,
 'Patients having sideway curvature of spine':4.0,
 'Patients with Diabetes':5.0,
 'Throat Cancer':6.0,
 'Children with cancer':2.0,
 'Chronic Kidney Disease':8.0,
 'Nueromuscular Disease':9.0,
 'Women with Cervical Cancer':10.0,
 ' Type 1 Diabetes Patients':11.0,
 'Disease that damage and weakens the muscle':12.0,
 'Genetic Disorder of Nervous System':13.0,
 'Groups of Men Surviving HIV':14.0,
 'Groups of Mothers of Children with Acute Myeloid Leukemia':15.0,
 'Groups of Surviving Women with HIV':16.0,
 'Patient Suffering with Arthritis':17.0,
 'Patient with Hepatitis':18.0,
 'Patients with Psoriasis':19.0,
 'Patients with Rheumatoid':20.0,
 'Ballooning of blood vessel in the brain':21.0,
 'Caring for Senior Citizen':22.0,
 'Chronic Myeloid Leukemia':23.0,
 'Disoder on muscle and tone posture causing damage to brain':14.0,
 'Hearing Disability':25.0,
 'Lack of sufficient blood Clothing':26.0,
 'Skin Diseases with too much protein collagen':27.0,
 'Type of Arthritis with a long term inflammation of joints and spine':28.0,
 'Women with Breast Cancer':29.0,
 'RARE Cancers':30.0}

In [ ]:
dfcp.show()

In [121]:
(traindfcp,testdfcp) = dfcp.randomSplit((0.7,0.3),seed=42)

In [124]:
from pyspark.ml.classification import LogisticRegression

In [126]:
lr = LogisticRegression(featuresCol='vectorizedFeatures',labelCol='label')

In [127]:
from pyspark.ml import Pipeline

In [128]:
pipeline = Pipeline(stages=[tokenizer,stopwords_remover,vectorizer,idf,lr])

In [129]:
pipeline

Pipeline_f05ddfd3a756

In [130]:
pipeline.stages

Param(parent='Pipeline_f05ddfd3a756', name='stages', doc='a list of pipeline stages')

In [131]:
lr_model = pipeline.fit(traindfcp)
lr_model

PipelineModel_bf778fe74243

In [132]:
predictions = lr_model.transform(testdfcp)
predictions.show()

+----------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Patient ID|             Illness|label|            mytokens|     filtered_tokens|         rawFeatures|  vectorizedFeatures|       rawPrediction|         probability|prediction|
+----------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|         3|Women with Cervic...| 10.0|[women, with, cer...|[women, cervical,...|(69,[0,8,60],[1.0...|(69,[0,8,60],[1.3...|[-0.3273996121341...|[1.01503324984236...|      10.0|
|         7|Patient with Hepa...| 18.0|[patient, with, h...|[patient, hepatitis]|(69,[23,53],[1.0,...|(69,[23,53],[3.03...|[-0.0635428384457...|[3.04387666964660...|      18.0|
|         9|Nueromuscular Dis...|  9.0|[nueromuscular, d...|[nueromuscular, d...|(69,[6,41],[1.0,1...|(69,[6,41],[2

In [133]:
predictions.columns

['Patient ID',
 'Illness',
 'label',
 'mytokens',
 'filtered_tokens',
 'rawFeatures',
 'vectorizedFeatures',
 'rawPrediction',
 'probability',
 'prediction']

In [134]:
predictions.select('rawPrediction','probability','Illness','label','prediction').show(10)

+--------------------+--------------------+--------------------+-----+----------+
|       rawPrediction|         probability|             Illness|label|prediction|
+--------------------+--------------------+--------------------+-----+----------+
|[-0.3273996121341...|[1.01503324984236...|Women with Cervic...| 10.0|      10.0|
|[-0.0635428384457...|[3.04387666964660...|Patient with Hepa...| 18.0|      18.0|
|[-0.0333629144949...|[3.05125971356492...|Nueromuscular Dis...|  9.0|       9.0|
|[0.13069916466009...|[1.49096151455236...|Type of Arthritis...| 28.0|      28.0|
|[-0.0739600613514...|[3.56248597452229...|      Blood Disorder|  1.0|       1.0|
|[-0.1530430574858...|[2.27582841413060...|Patient Suffering...| 17.0|      17.0|
|[21.8018645004919...|[0.99999999490446...|          Depression|  0.0|       0.0|
|[-0.0350973477811...|[1.37774083666522...|Lack of sufficien...| 26.0|      26.0|
|[21.8018645004919...|[0.99999999490446...|          Depression|  0.0|       0.0|
|[-0.32739961213

In [135]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [136]:
evaluator = MulticlassClassificationEvaluator(labelCol='label',predictionCol='prediction',metricName='accuracy')

In [137]:
accuracy = evaluator.evaluate(predictions)

In [138]:
accuracy

1.0

In [140]:
modelPath = "models/KNavigatorclassifiermodel"
lr_model.save(modelPath)

Py4JJavaError: ignored